In [ ]:
!git clone https://github.com/AhsanBaidar/MuLA_GAN.git

Cloning into 'MuLA_GAN'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 125 (delta 30), reused 78 (delta 15), pack-reused 28 (from 1)
Receiving objects: 100% (125/125), 144.94 MiB | 13.15 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
%cd MuLA_GAN

/content/MuLA_GAN


In [ ]:
%%writefile configs/train_MuLA-GAN.yaml
# --- Training Configuration for MuLA-GAN on UFO-120 ---

# Dataset Info
dataset_path: '/content/drive/My Drive/UFO-120/' # Base path (can be used relatively)
TRAIN_INPUT: 'train_val/lrd'    # <<< CORRECTED: Relative path to blurry train
TRAIN_GT: 'train_val/hr'       # <<< CORRECTED: Relative path to sharp train
TEST_INPUT: 'TEST/lrd'          # <<< CORRECTED: Relative path to blurry test
TEST_GT: 'TEST/hr'             # <<< CORRECTED: Relative path to sharp test
dataset_name: "UFO-120"          # Updated dataset name

# Image Info
im_width: 256
im_height: 256
chans: 3

# Model Info
MODEL_NAME: 'MuLA_GAN'
GENERATOR: 'mula_gan_g'
DISCRIMINATOR: 'mula_gan_d'

# Training Params
BATCH_SIZE: 8           # Original Batch Size
NUM_EPOCHS: 200         # Total epochs (adjust as needed)
LR_G: 0.0002
LR_D: 0.0002
B1: 0.5
B2: 0.999
WEIGHT_DECAY: 0.0001
LAMBDA_L1: 100
LAMBDA_ADV: 1
LAMBDA_PERCEPTUAL: 10   # Ensure VGG is used in train script if > 0

# Checkpoint/Logging Params
ckpt_interval: 20
SAMPLE_INTERVAL: 1000
LOG_INTERVAL: 50
val_interval: 1000

# Output Paths
CHECKPOINT_DIR: 'checkpoints'
SAMPLE_DIR: 'samples'
RESULTS_PATH: 'results'

Overwriting configs/train_MuLA-GAN.yaml


In [ ]:
%%writefile utils/data_utils.py
import os
import glob
import random
import numpy as np
from PIL import Image
import torch # Added torch import
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import yaml # Need yaml to read the config inside the function

class Dataloader(Dataset):
    def __init__(self, root, dataset_name, transforms_=None, config_path="configs/train_MuLA-GAN.yaml"): # Added config_path default
        self.transform = transforms.Compose(transforms_)

        # --- Read config to get correct paths ---
        try:
            with open(config_path) as f:
                cfg = yaml.load(f, Loader=yaml.FullLoader)
            # Use keys from the config file we created
            input_folder_rel = cfg.get("TRAIN_INPUT", 'train/trainA') # Default if key missing
            gt_folder_rel = cfg.get("TRAIN_GT", 'train/trainB')       # Default if key missing
            base_path = cfg.get("dataset_path", root)                # Use dataset_path from config

        except Exception as e:
            print(f"Error reading config file {config_path} in Dataloader: {e}")
            # Fallback or error out if config fails
            raise FileNotFoundError(f"Could not read/parse paths from {config_path}")

        # Construct absolute paths
        # dataset_path in config IS the absolute base path already
        input_path = os.path.join(base_path, input_folder_rel)
        gt_path = os.path.join(base_path, gt_folder_rel)

        print("--- Dataloader Init ---")
        print(f"Base Path (from config): {base_path}")
        print(f"Relative Input Path (from config): {input_folder_rel}")
        print(f"Relative GT Path (from config): {gt_folder_rel}")
        print(f"Constructed Input Path: {input_path}")
        print(f"Constructed GT Path: {gt_path}")
        # ---------------------------

        # Use glob directly on the constructed absolute paths
        self.filesA = sorted(glob.glob(input_path + "/*.*"))
        self.filesB = sorted(glob.glob(gt_path + "/*.*"))

        print(f"Found {len(self.filesA)} input files.")
        print(f"Found {len(self.filesB)} GT files.")
        # --------------------------

        if not self.filesA or not self.filesB:
             print("Error: Did not find files in one or both directories. Check paths and contents.")
             # Set len to 0 to trigger the ValueError correctly in DataLoader
             self.len = 0
        elif len(self.filesA) != len(self.filesB):
            print(f"Warning: Mismatch in number of files! Input: {len(self.filesA)}, GT: {len(self.filesB)}. Using minimum.")
            self.len = min(len(self.filesA), len(self.filesB))
        else:
            self.len = len(self.filesA)

        # Ensure self.len is not 0 before proceeding
        if self.len == 0:
             print("Critical Error: Dataset length is 0. Cannot train.")
             # Optionally raise an error here if preferred over letting DataLoader handle it
             # raise ValueError("Dataset length is 0 after checking files.")

        print(f"Setting dataset length to: {self.len}")
        print("-----------------------")


    def __getitem__(self, index):
        if self.len == 0:
             raise IndexError("Dataset is empty, cannot get item.")
        # Ensure index is within bounds (though % should handle it)
        actual_index = index % self.len

        # Add error checking for file reading
        try:
            img_A_path = self.filesA[actual_index]
            img_B_path = self.filesB[actual_index]
            img_A = Image.open(img_A_path).convert('RGB') # Ensure RGB
            img_B = Image.open(img_B_path).convert('RGB') # Ensure RGB

            # Optional: Check if filenames match (helps catch misaligned pairs)
            # base_A = os.path.basename(img_A_path).split('.')[0]
            # base_B = os.path.basename(img_B_path).split('.')[0]
            # if base_A != base_B:
            #     print(f"Warning: Filename mismatch at index {actual_index}: {base_A} vs {base_B}")

        except Exception as e:
            print(f"Error opening image at index {actual_index} (Paths: {self.filesA[actual_index]}, {self.filesB[actual_index]}): {e}")
            # Return dummy data or raise error? Dummy data might hide issues.
            dummy_tensor = torch.zeros((3, 256, 256)) # Assuming 256x256 size
            return {"A": dummy_tensor, "B": dummy_tensor}


        # Apply transforms only if images were loaded successfully
        if np.random.random() < 0.5:
             # Ensure array conversion happens correctly
            try:
                img_A_np = np.array(img_A)
                img_B_np = np.array(img_B)
                # Check shapes before flip
                if len(img_A_np.shape) == 3 and len(img_B_np.shape) == 3:
                     img_A = Image.fromarray(img_A_np[:, ::-1, :], "RGB")
                     img_B = Image.fromarray(img_B_np[:, ::-1, :], "RGB")
                # else: print(f"Warning: Skipping flip for non-3D array at index {actual_index}")
            except Exception as e:
                print(f"Error during random flip at index {actual_index}: {e}")


        try:
            img_A = self.transform(img_A)
            img_B = self.transform(img_B)
        except Exception as e:
            print(f"Error transforming image at index {actual_index}: {e}")
            dummy_tensor = torch.zeros((3, 256, 256))
            return {"A": dummy_tensor, "B": dummy_tensor}


        return {"A": img_A, "B": img_B}

    def __len__(self):
        # print(f"Dataset __len__ called, returning: {self.len}") # Optional debug
        return self.len

    # --- get_file_paths function is NO LONGER USED by __init__ ---

Overwriting utils/data_utils.py


In [ ]:
!python train_MuLA_GAN.py

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100% 548M/548M [00:02<00:00, 248MB/s]
@@@@@@@@@@@@@@
--- Dataloader Init ---
Base Path (from config): /content/drive/My Drive/UFO-120/
Relative Input Path (from config): train_val/lrd
Relative GT Path (from config): train_val/hr
Constructed Input Path: /co

In [ ]:
# --- IMPORTANT: ---
# 1. Replace 'UFO-120' if your checkpoint folder has a different name.
# 2. Replace 'generator_199.pth' with the actual highest epoch number you found.

!python test.py \
  --weights_path "checkpoints/UFO-120/generator_300.pth" \
  --data_dir "/content/drive/My Drive/UFO-120/TEST/lrd/" \
  --sample_dir "./output_ufo120/" # Using a new output folder name

Loaded model from checkpoints/UFO-120/generator_300.pth
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f0.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f10.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f11.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f12.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f122.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f14.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f15.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f16.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f17.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f19.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f2.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f20.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f22.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f23.jpg
Tested: /content/drive/My Drive/UFO-120/TEST/lrd/set_f26.jpg
Tested: /content/drive/My Driv

In [ ]:
%%writefile Evaluation/measure_ssim_psnr.py
"""
# > Script for measuring quantitative performances in terms of
#   - Structural Similarity Metric (SSIM)
#   - Peak Signal to Noise Ratio (PSNR)
# > Maintainer: https://github.com/xahidbuffon
"""
## python libs
import numpy as np
from PIL import Image
from glob import glob
from os.path import join, exists # Added exists
from ntpath import basename
import os
## local libs
# Make sure imqual_utils is importable
try:
    # Assumes running from MuLA_GAN root
    from Evaluation.imqual_utils import getSSIM, getPSNR
except ImportError:
    # Try importing from current directory if run from Evaluation/
    try:
         from imqual_utils import getSSIM, getPSNR
    except ImportError as e:
        print(f"Error importing imqual_utils: {e}")
        print("Please ensure you are running this script from the MuLA_GAN root directory or that Evaluation is in your Python path.")
        exit()


## compares avg ssim and psnr
def SSIMs_PSNRs(gtr_dir, gen_dir, im_res=(256, 256)):
    """
        - gtr_dir contain ground-truths
        - gen_dir contain generated images
    """
    gtr_paths = sorted(glob(join(gtr_dir, "*.*")))
    # Don't sort gen_paths here, look them up based on gtr_paths
    # gen_paths_all = sorted(glob(join(gen_dir, "*.*")))

    print(f"Found {len(gtr_paths)} potential ground truth images in: {gtr_dir}")
    print(f"Looking for corresponding generated images in: {gen_dir}")

    ssims, psnrs = [], []
    processed_count = 0

    if not gtr_paths:
        print(f"Error: No image files found in ground truth directory: {gtr_dir}")
        return np.array([]), np.array([])

    for gtr_path in gtr_paths:
        # Get base filename without extension
        gtr_f_base = basename(gtr_path).split('.')[0]
        # Construct expected generated file path (try common extensions)
        gen_path_expected_png = join(gen_dir, gtr_f_base + ".png")
        gen_path_expected_jpg = join(gen_dir, gtr_f_base + ".jpg")
        gen_path_expected_jpeg = join(gen_dir, gtr_f_base + ".jpeg")

        gen_path = None
        if exists(gen_path_expected_png):
            gen_path = gen_path_expected_png
        elif exists(gen_path_expected_jpg):
            gen_path = gen_path_expected_jpg
        elif exists(gen_path_expected_jpeg):
            gen_path = gen_path_expected_jpeg
        # Add more extensions if needed

        if gen_path: # Check if a corresponding file was found
            processed_count += 1
            # print(f"Processing pair: GT={gtr_path}, GEN={gen_path}") # Debug
            try:
                r_im = Image.open(gtr_path).resize(im_res)
                g_im = Image.open(gen_path).resize(im_res)

                # Ensure images are RGB before SSIM
                if r_im.mode != 'RGB': r_im = r_im.convert('RGB')
                if g_im.mode != 'RGB': g_im = g_im.convert('RGB')

                # get ssim on RGB channels
                ssim_val = getSSIM(np.array(r_im), np.array(g_im))
                if np.isfinite(ssim_val): ssims.append(ssim_val)

                # get psnr on L channel (grayscale)
                r_im_L = r_im.convert("L")
                g_im_L = g_im.convert("L")
                psnr_val = getPSNR(np.array(r_im_L), np.array(g_im_L))
                if np.isfinite(psnr_val): psnrs.append(psnr_val)

            except Exception as e:
                 print(f"Error processing {basename(gtr_path)} / {basename(gen_path)}: {e}")
        # else:
            # print(f"Skipping {basename(gtr_path)}: Corresponding generated file not found.") # Debug


    if processed_count == 0:
        print("\nError: No matching image pairs found between the two directories!")
        print("Please check paths and ensure filenames (without extension) are identical.")
        return np.array([]), np.array([]) # Return empty arrays

    return np.array(ssims), np.array(psnrs)


# --- Define YOUR paths here ---
# These paths point to your UFO-120 split data
gtr_dir = "/content/drive/My Drive/UFO-120/TEST/hr/"  # <<< Path to UFO-120 test ground truth
gen_dir = "./output_ufo120/"                          # <<< Path to your generated UFO-120 test images
# -----------------------------


### compute SSIM and PSNR
SSIM_measures, PSNR_measures = SSIMs_PSNRs(gtr_dir, gen_dir)

# Check if any valid results were returned before calculating mean/std
if len(SSIM_measures) > 0:
    print ("\n--- Results ---")
    print ("SSIM on {0} matched samples".format(len(SSIM_measures)))
    print ("Mean: {0:.4f} std: {1:.4f}".format(np.mean(SSIM_measures), np.std(SSIM_measures)))
else:
    print("\n--- No valid SSIM scores calculated. ---")


if len(PSNR_measures) > 0:
    print ("\nPSNR on {0} matched samples".format(len(PSNR_measures)))
    print ("Mean: {0:.2f} std: {1:.2f}".format(np.mean(PSNR_measures), np.std(PSNR_measures)))
else:
     print("\n--- No valid PSNR scores calculated. ---")

if len(SSIM_measures) == 0 and len(PSNR_measures) == 0:
    print ("\n--- No results calculated. Check paths, filenames, and potential errors during processing. ---")

Overwriting Evaluation/measure_ssim_psnr.py


In [ ]:
!python Evaluation/measure_ssim_psnr.py

Found 120 potential ground truth images in: /content/drive/My Drive/UFO-120/TEST/hr/
Looking for corresponding generated images in: ./output_ufo120/

--- Results ---
SSIM on 120 matched samples
Mean: 0.7898 std: 0.0518

PSNR on 120 matched samples
Mean: 26.83 std: 2.79


In [ ]:
%%writefile Evaluation/measure_uiqm.py
"""
# > Script for measuring quantitative performance in terms of UIQM
# > Maintainer: https://github.com/xahidbuffon
"""
## python libs
import numpy as np
from PIL import Image, ImageOps
from glob import glob
from os.path import join
from ntpath import basename
import os # Added os import
## local libs
# Make sure uqim_utils is importable
try:
    # Assumes running from MuLA_GAN root
    from Evaluation.uqim_utils import getUIQM
except ImportError:
    # Try importing from current directory if run from Evaluation/
    try:
         from uqim_utils import getUIQM
    except ImportError as e:
        print(f"Error importing uqim_utils: {e}")
        print("Please ensure you are running this script from the MuLA_GAN root directory or that Evaluation is in your Python path.")
        exit()


def measure_UIQMs(dir_name, im_res=(256, 256)):
    paths = sorted(glob(join(dir_name, "*.*")))
    print(f"Found {len(paths)} files in: {dir_name}") # Debug print
    uqims = []
    i=0
    if not paths:
        print(f"Error: No images found in directory: {dir_name}")
        return np.array([])

    for img_path in paths:
        # print(f"Processing image {i+1}/{len(paths)}: {basename(img_path)}") # Debug print
        i=i+1
        try:
            im = Image.open(img_path).resize(im_res)
             # Ensure image is RGB for UIQM calculation
            if im.mode != 'RGB': im = im.convert('RGB')
            im_array = np.array(im)
            # print(im_array.shape) # Debug print shape
            uiqm = getUIQM(im_array)
            if np.isfinite(uiqm): # Check for NaN/Inf
                 uqims.append(uiqm)
            # else:
                # print(f"  -> Warning: UIQM calculation resulted in NaN/Inf for {basename(img_path)}")

        except Exception as e:
             print(f"Error processing {basename(img_path)}: {e}")

    if not uqims:
        print("\nError: UIQM calculation failed for all images.")
        return np.array([])

    return np.array(uqims)


# --- Define YOUR path here ---
# This path points to your generated UFO-120 results
gen_dir = "./output_ufo120/" # <<< UPDATED
# -----------------------------

# UIQMs of the enhanceded output images
gen_uqims = measure_UIQMs(gen_dir)

if len(gen_uqims) > 0: # Only print if results were calculated
    print ("\n--- Results ---")
    print ("UIQM on {0} samples".format(len(gen_uqims)))
    print ("Mean: {0:.4f} std: {1:.4f}".format(np.mean(gen_uqims), np.std(gen_uqims)))
else:
     print ("\n--- No UIQM results calculated. Check path and logs. ---")

Overwriting Evaluation/measure_uiqm.py


In [ ]:
!python Evaluation/measure_uiqm.py

Found 120 files in: ./output_ufo120/

--- Results ---
UIQM on 120 samples
Mean: 2.7016 std: 0.5266
